# Generazione dati progetto di laboratorio

## Importo le librerie

In [1]:
import pandas as pd
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine, text, MetaData
import random 
import datetime
from pprint import pprint
import random
fake = Faker('it-IT')

In [2]:
NUM_FORNITORI = 1000
NUM_DIPENDENTI = 10000
NUM_MATRIMONI = 100
NUM_PROGETTI = 5000
NUM_COMPETENZE_DIP = 10
NUM_DIP_PROGETTO = 5

FILE_DIPARTIMENTO = "MOCK_DEPARTMENT.txt"
FILE_QUALIFICHE = "MOCK_QUALIFY.txt"
FILE_SKILLS = "MOCK_SKILLS.txt"
FILE_LAUREE = "MOCK_DEGREES.txt"
FILE_DOTTORATI = "MOCK_PHDS.txt"

In [3]:


#FORNITORE
fornitore = defaultdict(list)
for _ in range(NUM_FORNITORI):
    fornitore["nome"].append(fake.company())
    fornitore["indirizzo"].append(fake.address())

df_fornitore = pd.DataFrame(fornitore)
df_fornitore.drop_duplicates(inplace=True, subset="nome")

print("Created Fornitore")

print(df_fornitore)


Created Fornitore
                                   nome  \
0            Caccioppoli-Mantegna Group   
1            Abatantuono-Beccaria Group   
2             Curatoli-Satriani e figli   
3                        Saraceno Group   
4                        Milanesi Group   
..                                  ...   
995                Farnese-Bacosi Group   
996  Bertoli, Asprucci e Gianetti Group   
997                       Cugia e figli   
998                        Pugliese SPA   
999  Boccherini, Pometta e Bembo s.r.l.   

                                             indirizzo  
0    Strada Smirnoff, 563 Appartamento 5\n16163, Sa...  
1                Vicolo Onisto, 0\n31041, Cornuda (TV)  
2         Incrocio Gioachino, 62\n36071, Pugnello (VI)  
3       Stretto Ugo, 28 Piano 1\n97019, Scoglitti (RG)  
4       Canale Patrizia, 925 Piano 8\n00126, Roma (RM)  
..                                                 ...  
995    Via Fibonacci, 11\n17051, Marina Di Andora (SV)  
996      

In [4]:



#DIPARTIMENTO

dipartimento = defaultdict(list)

f = open(FILE_DIPARTIMENTO)
lines = f.read().splitlines()
f.close()

dipartimento["recapito"] = (fake.random_sample(elements = range(100000,999999), length = len(lines)))

for i in range(len(lines)):
    dipartimento["recapito"][i] = int("0432" + str(dipartimento["recapito"][i]))

for line in lines:
    dipartimento["nome"].append(line)
    dipartimento["email"].append(line.replace(" ","").lower() + "@company.com")
    dipartimento["ultimo_acquisto"].append(random.choice(fornitore["nome"]))
    dipartimento["data_ultimo_acquisto"].append(fake.date_between(start_date = datetime.date(2021,1,1)))

#Numero afferenti viene aggiunto dopo

print("Created Dipartimento")

df_dipartimento = pd.DataFrame(dipartimento)

print(df_dipartimento)

df_dipartimento.to_csv("dipartimento.csv")


Created Dipartimento
     recapito                         nome  \
0   432512453               Administration   
1   432207852      Artificial Intelligence   
2   432264584     Branding and Advertising   
3   432313672             Brand Management   
4   432793394           Business Analytics   
..        ...                          ...   
93  432189137              User Experience   
94  432137475            Vendor Management   
95  432991760   Warehouse and Distribution   
96  432723057              Web Development   
97  432810050            Workplace Culture   

                                   email  \
0             administration@company.com   
1     artificialintelligence@company.com   
2     brandingandadvertising@company.com   
3            brandmanagement@company.com   
4          businessanalytics@company.com   
..                                   ...   
93            userexperience@company.com   
94          vendormanagement@company.com   
95  warehouseanddistribution@c

In [5]:
#FORNISCE

fornisce = defaultdict(list)

for _ in range(400):
    i = random.randint(0,len(dipartimento["nome"])-1)
    p = random.randint(0,9)
    fornisce["dipartimento"].append(dipartimento["nome"][i])
    if p < 9:
        fornisce["fornitore"].append(dipartimento["ultimo_acquisto"][i])
    else:
        fornisce["fornitore"].append(fake.random_element(fornitore["nome"]))


print("Created Fornisce")

df_fornisce = pd.DataFrame(fornisce)
df_fornisce.drop_duplicates(inplace=True)

pprint(df_fornisce)


Created Fornisce
                     dipartimento                         fornitore
0                           Sales                   Onio-Polani SPA
1              Regulatory Affairs                      Marcacci SPA
2                Event Management  Mascagni, Abba e Dallara e figli
3                   Cybersecurity                 Pascarella s.r.l.
4              Supplier Relations                  Bernardi e figli
..                            ...                               ...
374              Customer Service                   Benigni e figli
380        International Business                   Rossini e figli
382                        Design            Antonacci-Venier Group
390          Software Engineering                    Berlusconi SPA
396   Online Community Management                       Calarco SPA

[147 rows x 2 columns]


In [6]:
#DIPENDENTE

dipendente = defaultdict(list)

dipendente["matricola"] = fake.random_sample(elements = range(1, 1000000), length = NUM_DIPENDENTI)

f = open(FILE_QUALIFICHE)
qualifies = f.read().splitlines()
f.close()

f = open(FILE_LAUREE)
degrees = f.read().splitlines()
f.close()

f = open(FILE_DOTTORATI)
phds = f.read().splitlines()
f.close()




for _ in range(NUM_DIPENDENTI):
    dipendente["nome"].append(fake.first_name())
    dipendente["cognome"].append(fake.last_name())
    dipendente["qualifica"].append(fake.random_element(elements=qualifies))
    dipendente["dipartimento"].append(fake.random_element(elements=dipartimento["nome"]))
    dipendente["data_assunzione"].append(fake.date_time_between(start_date = datetime.date(2015, 1, 1)))

    nascita = fake.date_time_between(end_date = datetime.date(2003, 1, 1))
    dipendente["data_di_nascita"].append(nascita)

    p_laureato = random.randint(0,9)
    if p_laureato < 5:
        dipendente["classe_laurea"].append(fake.random_element(elements = degrees))
        dipendente["data_laurea"].append(fake.date_time_between(start_date = nascita + datetime.timedelta(days = 20*365)))

        p_dottorato = random.randint(0,9)
        if p_dottorato < 2:
            dipendente["classe_dottorato"].append(fake.random_element(elements = phds))
            dipendente["data_dottorato"].append(fake.date_time_between(start_date = nascita + datetime.timedelta(days = 28*365)))
        else:
            dipendente["classe_dottorato"].append("")
            dipendente["data_dottorato"].append(None)
    else:
        dipendente["classe_laurea"].append("")
        dipendente["data_laurea"].append(None)
        dipendente["classe_dottorato"].append("")
        dipendente["data_dottorato"].append(None)


df_dipendente = pd.DataFrame(dipendente)

print("Created Dipendente")

df_dipartimento.insert(2, "numero_afferenti", 0)

#df_dipartimento.set_index("nome", inplace=True)

for i in range(len(df_dipendente.index)):
    dip = df_dipendente.iloc[[i]]["dipartimento"].to_string(index = False)
    df_dipartimento.loc[df_dipartimento["nome"] == dip, "numero_afferenti"] +=  1

pprint(df_dipendente)
pprint(df_dipartimento["numero_afferenti"].sum())

Created Dipendente
      matricola        nome    cognome  \
0         40687      Bianca  Ostinelli   
1        907145    Vittorio     Priuli   
2        236086      Giosuè  Morandini   
3         31474  Alessandro    Vezzali   
4        629799     Gastone   Doglioni   
...         ...         ...        ...   
9995     577319     Ugolino  Impastato   
9996     807687     Cirillo   Virgilio   
9997     188226     Tiziana     Sraffa   
9998     634540    Maurizio   Sismondi   
9999     620382     Gaspare  Bajamonti   

                                              qualifica  \
0                          Adaptability and flexibility   
1     Ability to develop and execute social media st...   
2                                  Leadership qualities   
3          Experience with product lifecycle management   
4     Strong understanding of inventory management p...   
...                                                 ...   
9995  Strong understanding of organizational develop...   
9996

In [7]:
df_dipendente.to_csv("dipendente.csv")
df_dipartimento.to_csv("dipartimento.csv")

In [8]:
#Matrimonio

matrimonio = defaultdict(list)

coniugi = fake.random_sample(elements = dipendente["matricola"], length = NUM_MATRIMONI*2)

for i in range(NUM_MATRIMONI):
    matrimonio["coniuge1"].append(coniugi[2*i])
    matrimonio["coniuge2"].append(coniugi[2*i+1])

print("Created Matrimonio")

df_matrimonio = pd.DataFrame(matrimonio)

print(df_matrimonio)

Created Matrimonio
    coniuge1  coniuge2
0     267822    539045
1     951986    628381
2     120057    354581
3     399309    941054
4     427651    339745
..       ...       ...
95    359186      8277
96    834118    351865
97    543075    483782
98    268717    103455
99    347151    213443

[100 rows x 2 columns]


In [9]:
#PROGETTO

progetto = defaultdict(list)

progetto["codice_aziendale"] = fake.random_sample(elements = range(1000000), length = NUM_PROGETTI)
progetto["budget"] = fake.random_elements(elements = range(1000, 50000), unique = False, length = NUM_PROGETTI)
progetto["durata_in_mesi"] = fake.random_elements(elements = range(6, 48), unique = False, length=NUM_PROGETTI)

print("Created Progetto")

df_progetto = pd.DataFrame(progetto)

print(df_progetto)


Created Progetto
      codice_aziendale  budget  durata_in_mesi
0               226948   16517              31
1                56338   31386              21
2               744637   10420              14
3                18457   25195              26
4               368122   20892              20
...                ...     ...             ...
4995            934752   29014              20
4996              3929   29308              24
4997            208719   44145              30
4998            281135   14752              46
4999            715694   24383              17

[5000 rows x 3 columns]


In [10]:
#COMPETENZA

competenza = defaultdict(list)


f = open(FILE_SKILLS)
skills = f.read().splitlines()
f.close()

competenza["nome"] = skills

print("Created Competenza")

df_competenza = pd.DataFrame(competenza)
df_competenza.drop_duplicates(inplace=True)

print(df_competenza)


Created Competenza
                                    nome
0                          Communication
1                          Collaboration
2                           Adaptability
3                        Problem-solving
4                        Time management
..                                   ...
95                   Emotional stability
96                           Mindfulness
97  Coaching for performance improvement
98                         Time-blocking
99                 Feedback solicitation

[97 rows x 1 columns]


In [11]:
#POSSIEDE

possiede = defaultdict(list)

def add_possiede(i):
    possiede["competenza"].append(fake.random_element(competenza["nome"])) 
    possiede["matricola"].append(fake.random_element(dipendente["matricola"]))    
    if(i % 10000 == 0):
        print(i)


for i in range(NUM_COMPETENZE_DIP*NUM_DIPENDENTI):
    add_possiede(i)

df_possiede = pd.DataFrame(possiede)
df_possiede.drop_duplicates(inplace = True)


print("Created Possiede")

print(df_possiede)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Created Possiede
                                     competenza  matricola
0                            Consensus-building     975131
1                               Active learning     211222
2                                    Creativity     492782
3                          Cultural integration     338997
4                                    Resilience     307105
...                                         ...        ...
99995                          Active listening     343841
99996                                Persuasion     794502
99997                     Relationship building     100220
99998                            Self-awareness     835320
99999  Decision-making with limited information     483796

[94839 rows x 2 columns]


In [12]:
#COINVOLGE

coinvolge = defaultdict(list)

for p in progetto["codice_aziendale"]:
    matr = fake.random_sample(dipendente["matricola"], length = NUM_DIP_PROGETTO)
    for i in range(NUM_DIP_PROGETTO):
        competenze  = df_possiede.loc[df_possiede['matricola'] == matr[i]]["competenza"].tolist()
        if(competenze != []):
            coinvolge["matricola"].append(matr[i])
            coinvolge["competenza"].append(fake.random_element(competenze))
            coinvolge["progetto"].append(p)
        #coinvolge["competenza"].append(fake.random_element(posssiede))
        
print("Created Coinvolge")  

df_coinvolge = pd.DataFrame(coinvolge)

print(df_coinvolge)

Created Coinvolge
       matricola               competenza  progetto
0         309590     Emotional regulation    226948
1         375615      Conflict management    226948
2         268463  Giving credit to others    226948
3         496475     Interpersonal skills    226948
4         876293     Cultural sensitivity    226948
...          ...                      ...       ...
24993     840796          Open-mindedness    715694
24994     927668          Open-mindedness    715694
24995     601839               Delegation    715694
24996     656782       Influencing skills    715694
24997     639120          Open-mindedness    715694

[24998 rows x 3 columns]


In [13]:
engine = create_engine("postgresql+psycopg2://postgres:postgres@/progetto_basi")

with engine.connect() as con:
    f = open("data_definition.sql")
    data_definition = text(f.read())
    f.close()

    f = open("trigger.sql")
    trigger = text(f.read())
    f.close()

    f = open("indexes.sql")
    indici = text(f.read())
    f.close()
    
    con.execute(data_definition)
    con.execute(trigger)
    con.execute(indici)
    con.commit()

In [15]:

df_fornitore.to_sql('fornitore', con=engine, index=False, if_exists='append')
df_dipartimento.to_sql('dipartimento', con=engine, index=False, if_exists='append')
df_fornisce.to_sql('fornisce', con=engine, index=False, if_exists='append')
df_dipendente.to_sql('dipendente', con=engine, index=False, if_exists='append')
df_matrimonio.to_sql('matrimonio', con=engine, index=False, if_exists='append')
df_progetto.to_sql('progetto', con=engine, index=False, if_exists='append')
df_competenza.to_sql('competenza', con=engine, index=False, if_exists='append')
df_possiede.to_sql('possiede', con=engine, index=False, if_exists='append')
df_coinvolge.to_sql('coinvolge', con=engine, index=False, if_exists='append')

998